**Example 2 - RAG on medical coding spreadsheet using langchain and Chroma**

In [1]:
! pip install openai langchain_community langchain langchain_openai chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
AZURE_OPENAI_ACCOUNT: str = ""
AZURE_OPENAI_KEY: str = ""

In [4]:
from openai import AzureOpenAI

# Set up clients and specify the chat model
openai_client = AzureOpenAI(
     api_version="2024-06-01",
     azure_endpoint=AZURE_OPENAI_ACCOUNT,
     api_key=AZURE_OPENAI_KEY
 )

deployment_name = "gpt-4o"

In [5]:
prompt = '''
You are an expert medical assistant who extracts a comprehensive list of all possible diagnoses, comorbidies and abnormalities from patient discharge summaries. The discharge summaries are in markdown and may contain tables and additional metadata.
Give only diagnoses that can be inferred solely based on the information I have given - do not use any other information.
Bear in mind that the list of diagnoses you give will be used by medical coders later on so make the diagnoses as detailed and specific as possible given the available patient data.
Before finalising your answer check if you haven't missed any abnormal data points and hence any diagnoses that could be made based on them. If you did, add them to your response.
Start your answer with your list of diagnoses and do not generate any other text apart from the text needed to list the diagnoses.
'''

In [6]:
dc = '''
<figure>

![](figures/0)

<!-- FigureContent="\+ Indraprastha Apollo Hospitals touching lives DELHI -" -->

</figure>


DISCHARGE SUMMARY
===

|||
| - | - |
| NAME | : MASTER.ANKIT BISWASH |
| UHID | : APD1.0000807307 |

|||
| - | - |
| Age/Sex | : 3 Year(s) Male |
| IPNO | : 208812 |

Date of Admission : 22-Nov-2008

Room/Ward

: 4159 (Recommended-Govt Of Delhi)

Date of Discharge :01-Apr-2009

Consultants

1\. Dr.A.K. Banerji NEUROSURGEON 2. Dr. Varindera Paul Singh NEUROSURGEON

Unit :NEUROSURGERY

DIAGNOSIS : Posterior Third Ventricle Ependymoma Hydrocephalus

HISTORY :- Right side facial weakness - 1 month Headache & vomiting - 20 days Difficulty in walking & frequent falls - 15 days CECT head revealed posterior third ventricle tumor with hydrocephalus

ON EXAMINATION

Child - sick BP - 100/60 mm Pupil - B/L 3 mm, reacting Fundus - B/L mild papilloedema Moving all four limbs

COURSE IN HOSPITAL :-

Pre-operatively was evaluated by Dr Nameet Jerath [Senior Consultant - Paediatric], and was taken up for surgery after his clearance.

OPERATIVE PROCEDURE: Midline suboccipital craniectomy, infratentorial-supracerebellar approach and microsurgical

Contd .. 2 ...

<figure>

![](figures/1)

<!-- FigureContent="P Joint Commission INTERNATIONAL" -->

</figure>


India's First Internationally Accredited Hospital Indraprastha Apollo Hospitals Sarita Vihar, Delhi - Mathura Road, New Delhi - 110 076 (INDIA) Tel. : 91-11- 26925858. 26925801, Fax : 91-11-26823629, Emergency Telephone No. : 1066. Website : www.apollohospdelhi.com
<figure>

![](figures/2)

<!-- FigureContent="Indraprastha Apollo Hospitals touching lives DELHI -" -->

</figure>


tumor decompression done under GA on 14.11.08.

Right parieto-occipital burrhole made and external ventricular drain inserted.

FINDINDS: Soft mildly vascular greyish tumor in posterior third ventricle.

HISTOPATHOLOGY: Ependymoma ( WHO - Gr II, Tanycytic type)

POST-OPERATIVE HOSPITAL STAY:

Was electively ventilated post-operativley in Neuro ICU. Next day weaned off from ventilator & extubated. NCCT head done on 15.11.08 ruled out any operative site haematoma with surgical changes and decrease in mass effect. Developed seizure followed by cardiac arrest. Recovered after resucitation & modification of anticonvulsant drugs. Kept on Midazolam infusion & gradually tapered off & shifted to Paed. ICU. NCCT head on 19.11.08, after removal of external ventricular drain revealed decrease in size of ventricles. Had copious throat secretions, so put on ventilatory support for 48 hour & then weaned off. Improve gradually & shifted out to Govt.ICU ward on 22.11.08. Dr. D.K.Mahajan [ Senior Consultant - Dermatology ] opinion taken for allergic skin lesion. Dr. S. Haldhar [Consultant -Radiation Oncology] opinion was taken for post operative adjuvant radiotherapy. Developed loose motion & fever, was managed with help of Paediatrician.

Contrast MRI brain done on 16.12.08 revealed surgical changes with persistent ventriculomegaly.

OPERATIVE PROCEDURE:

Right VP shunt [Medium Pressure - Pudenz] done under done under GA on 01.01.2009 FINDINGS: Clear CSF came in moderate pressure.

POST-OPERATIVE HOSPITAL STAY

Had mild improvement in the tonic posturing after the shunt surgery. IMRT was started in consultation with Dr Shikha Haldhar (Consultant Radiation Oncology). Had recurrent seizures with respiratory distress which was managed in consultation with Dr. N.Jerath (Senior Consultant,Pediatrics) and Dr. V.B.Gupta( Senior Consultant, Pediatric Neurology). Follow up NCCT head and MRI brain done revealed residual tumour in the region of posterior third ventricle with decreased ventricular size with increase in the bilateral subdural collections with no significant compression. Subdural tap was done and around 150cc of fluid aspirated, no neurological improvement seen after the procedure. Had recurrent episodes of laryngeal stridor. Dr. G. Raheja (Senior Consultant,ENT) was consulted who did tracheostomy on 23.03.2009. Follow up NCCT head done on 23.03.2009 revealed no change in the subdural collection.

COMMENTS:

Long term prognosis guarded. Needs regular follow up and neurorehabilitation.

CONDITION OF THE PATIENT AT THE TIME OF DISCHARGE :- BP - 100/60 mm

<!-- PageNumber="Contd .. 3 ..." -->

<figure>

![](figures/3)

<!-- FigureContent="Joint Commission INTERNATIONAL" -->

</figure>


India's First Internationally Accredited Hospital Indraprastha Apollo Hospitals Sarita Vihar, Delhi - Mathura Road, New Delhi - 110 076 (INDIA) Tel. : 91-11- 26925858, 26925801, Fax : 91-11-26823629, Emergency Telephone No. : 1066 Website : www.apollohospdelhi.com
<figure>

![](figures/4)

<!-- FigureContent="Indraprastha Apollo Hospitals touching lives - DELHI -" -->

</figure>


Afebrile Improving gaze paresis Opening eyes spontaneously, Tonic posturing present. Ryle's tube feed with oral supplement Wound healthy, Sutures removed On Tracheostomy

ADVICE ON DISCHARGE :-

DIET: Ryle's tube feeding 100 ml 2 hourly with oral supplement as tolerated

PHYSICAL ACTIVITY: Wheel chair mobilization

MEDICATIONS: Syp Augmentin 5ml thrice daily X 1 week Gardenal 60mgm twice daily X 1 month Syp Tegrital (100mgm /5ml) 4ml thrice daily X 1 month Frisium 5 mgm in morning and noon, and, 10mgm at bedtime X 1 month Tizan 2mgm thrice daily X 2 weeks Baclofen 7 mgm thrice daily X 2 weeks Diamox 250mgm half tablet twice daily X 2 weeks Rantac 25 mgm twice daily X 2 weeks Syp Visyneral 5ml daily X 2 weeks Asthalin and saline nebulization thrice daily X 1 month Crocin 10 ml SOS for fever/ headache Refresh eye drop both eye four hourly X 1 week

OTHERS:

Physiotherapy and mobilisation as advised Care of Tracheostomy and Ryle's tube

FOLLOW-UP: To follow up with Dr A K Banerji in Neurosciences OPD after 1 month Kindly confirm your appointment before coming. For appointments contact - 9910000409

For any medical problem contact Dr H S Sohal (9818535985)

Dr. A.K. Banerji SR.CONSULTANT NEUROSURGEON

Contd .. 4 ...

<figure>

![](figures/5)

<!-- FigureContent="Joint Commission INTERNATIONAL" -->

</figure>


India's First Internationally Accredited Hospital Indraprastha Apollo Hospitals Sarita Vihar, Delhi - Mathura Road, New Delhi - 110 076 (INDIA) Tel. : 91-11-26925858, 26925801, Fax : 91-11-26823629, Emergency Telephone No .: 1066 Website : www.apollohospdelhi.com

<!-- PageNumber="S2" -->
:selected: :selected: :selected: :selected:<figure>

![](figures/6)

<!-- FigureContent="Indraprastha Apollo Hospitals DELHI - touching lives" -->

</figure>


Dr. Varindera Paul Singh SR.CONSULTANT NEUROSURGEON

<figure>

![](figures/7)

<!-- FigureContent="30/1/09" -->

</figure>


\*HSS

Registrar/Resident/C.M.O

Date

: 30-Mar-2009 10:27 AM.

<figure>

![](figures/8)

<!-- FigureContent="Joint Commission INTERNATIONAL" -->

</figure>


India's First Internationally Accredited Hospital Indraprastha Apollo Hospitals Sarita Vihar, Delhi - Mathura Road, New Delhi - 110 076 (INDIA) Tel. : 91-11- 26925858, 26925801, Fax : 91-11-26823629, Emergency Telephone No. : 1066 Website : www.apollohospdelhi.com

<!-- PageNumber="S .-" -->
'''

In [7]:
diag = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    model=deployment_name
)

In [8]:
print(diag.choices[0].message.content)

1. Acute Myocardial Infarction
2. Type 2 Diabetes Mellitus
3. Hypertension
4. Chronic Kidney Disease, Stage 3
5. Hyperlipidemia
6. Congestive Heart Failure
7. Chronic Obstructive Pulmonary Disease (COPD)
8. Peripheral Artery Disease
9. Obesity (BMI 32)
10. Atrial Fibrillation
11. Anemia, unspecified
12. Depression
13. Osteoarthritis of the knee
14. Gastroesophageal Reflux Disease (GERD)


In [9]:
list_of_diagnosis = [d for d in diag.choices[0].message.content.split('\n') if d != '']
# remove numbers from the list
list_of_diagnosis = [d.split('. ')[1] for d in list_of_diagnosis] #might need to put :
list_of_diagnosis

['Acute Myocardial Infarction',
 'Type 2 Diabetes Mellitus',
 'Hypertension',
 'Chronic Kidney Disease, Stage 3',
 'Hyperlipidemia',
 'Congestive Heart Failure',
 'Chronic Obstructive Pulmonary Disease (COPD)',
 'Peripheral Artery Disease',
 'Obesity (BMI 32)',
 'Atrial Fibrillation',
 'Anemia, unspecified',
 'Depression',
 'Osteoarthritis of the knee',
 'Gastroesophageal Reflux Disease (GERD)']

In [11]:
data = "ICD11To10MapToOneCategory.csv"
# read csv file
import pandas as pd
df = pd.read_csv(data, encoding='unicode_escape')

In [12]:
df

,Linearization (release) URI,icd11Code,icd11Chapter,icd11Title,icd10Code,icd10Chapter,icd10Title,21/01/2024
0,http://id.who.int/icd/release/11/2024-01/mms/1...,01,1,Certain infectious or parasitic diseases,I,I,Certain infectious and parasitic diseases,NaN
1,http://id.who.int/icd/release/11/2024-01/mms/5...,NaN,1,Gastroenteritis or colitis of infectious origin,I,I,Certain infectious and parasitic diseases,NaN
2,http://id.who.int/icd/release/11/2024-01/mms/1...,NaN,1,Bacterial intestinal infections,A04.9,I,"Bacterial intestinal infection, unspecified",NaN
3,http://id.who.int/icd/release/11/2024-01/mms/2...,1A00,1,Cholera,A00.9,I,"Cholera, unspecified",NaN
4,http://id.who.int/icd/release/11/2024-01/mms/4...,1A01,1,Intestinal infection due to other Vibrio,A04,I,Other bacterial intestinal infections,NaN
...,...,...,...,...,...,...,...,...
17903,http://id.who.int/icd/release/11/2024-01/mms/4...,RA22,25,National emergency code 03,XXII,XXII,Codes for special purposes,NaN
17904,http://id.who.int/icd/release/11/2024-01/mms/2...,RA23,25,National emergency code 04,XXII,XXII,Codes for special purposes,NaN
17905,http://id.who.int/icd/release/11/2024-01/mms/1...,RA24,25,National emergency code 05,XXII,XXII,Codes for special purposes,NaN
17906,http://id.who.int/icd/release/11/2024-01/mms/1...,RA25,25,National emergency code 06,XXII,XXII,Codes for special purposes,NaN


In [13]:
df = df.loc[~df.icd11Code.isna(), :]

In [14]:
import torch
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_openai import AzureOpenAIEmbeddings

def embedding_func(selected_embedding: str = "OpenAIEmbeddings"):
    if selected_embedding == "OpenAIEmbeddings":

        import os
        os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
        os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ACCOUNT

        embeddings = AzureOpenAIEmbeddings(
            azure_deployment="text-embedding-ada-002",
            openai_api_version="2023-05-15",
        )

        embedding_func = embeddings

    elif selected_embedding == "HuggingFaceEmbeddings":
        device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
        embedding_func = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5", model_kwargs={"device": device})
    else:
      raise ValueError("uknown type")
    return embedding_func


def create_langchain_documents(df, target_col):
    """create langchain documents from data"""
    cols = df.columns
    return df.apply(lambda row: Document(page_content=row[target_col], metadata=row[cols].to_dict()), axis=1).tolist()


def create_rag(df):
    target_col = "icd11Title"

    # create documents from sample symptoms
    docs = create_langchain_documents(df, target_col)

    _embedding_func = embedding_func()
    vectorstore = Chroma.from_documents(documents=docs, embedding=_embedding_func)

    return df, docs, vectorstore

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [15]:
%%time
df, docs, vectorstore = create_rag(df)

CPU times: user 1min 16s, sys: 3.2 s, total: 1min 19s
Wall time: 2min 3s


In [16]:
for diagnosis in list_of_diagnosis:

    # semantic search (Vector store search)
    print(diagnosis)
    print('_______________________________________')
    result = vectorstore.similarity_search_with_score(diagnosis, k=3)
    report = pd.DataFrame([d.metadata for d, score in result])
    report = report.set_index('icd11Code')
    print(report[['icd11Title']])
    print()
    print()

Acute Myocardial Infarction
_______________________________________
                                         icd11Title
icd11Code                                          
BA41                    Acute myocardial infarction
BA41.Z     Acute myocardial infarction, unspecified
BA41.0     Acute ST elevation myocardial infarction


Type 2 Diabetes Mellitus
_______________________________________
                                        icd11Title
icd11Code                                         
5A11                      Type 2 diabetes mellitus
5A10                      Type 1 diabetes mellitus
5A13       Diabetes mellitus, other specified type


Hypertension
_______________________________________
                           icd11Title
icd11Code                            
BA01       Hypertensive heart disease
BA00           Essential hypertension
BB01           Pulmonary hypertension


Chronic Kidney Disease, Stage 3
_______________________________________
                               